In [ ]:
from __future__ import print_function

%matplotlib inline

import sys
import tensorflow as tf
from tensorflow.python.framework import ops

from collections import OrderedDict

import numpy as np
import scipy.misc
from input import *
from model import *
from train import *
import matplotlib.pyplot as plt
tf.reset_default_graph()

1.3.0


In [ ]:
image_shape = (160,576)
iterator, filename = get_train_inputs(batch_size=100,
                                      repeat=True, 
                                      num_classes=2, 
                                      image_shape=image_shape)

In [ ]:
_show = False
if _show:
    with tf.Session() as sess:
        sess.run(iterator.initializer, feed_dict={filename: ['data/kitti_segmentation.tfrecord']})
        next_element = iterator.get_next()
        i = 1
        while i < 5:
            i += 1
            print("*"*10)
            image, label = sess.run(next_element)
            new = scipy.misc.imresize(image,(160*2,576*2) )
            plt.imshow(image[:,:,:])
            plt.show()
            plt.imshow(new[:,:,:])
#             plt.imshow(label[:,:,0], cmap='jet', alpha=0.5)
            plt.show()
            print(new.shape, image.shape)

In [ ]:
# record_iterator = tf.python_io.tf_record_iterator(path='data/kitti_segmentation.tfrecord')
# string_record = next(record_iterator)
# example = tf.train.Example()
# example.ParseFromString(string_record)

## Model

In [ ]:
encoder = SlimModelEncoder(name="vgg_16", num_classes=2, is_training=True)
image, label = iterator.get_next()
init_fn, end_points = encoder.build(image=image, image_shape=image_shape)

In [ ]:
end_points.keys()

In [ ]:
print(end_points['vgg_16/conv3/conv3_3'], 
    end_points['vgg_16/conv4/conv4_3'], 
    end_points['vgg_16/conv5/conv5_3'],
    end_points['vgg_16/fc6'],
    end_points['vgg_16/fc7'],sep='\n')

In [ ]:
decoder = FCNDecoder(end_points, nb_classes=2, scope='decoder')

In [ ]:
tensors_to_connect = OrderedDict()
tensors_to_connect["vgg_16/fc7"] = (2,2)
tensors_to_connect['vgg_16/conv5/conv5_3'] = (2,2)
tensors_to_connect['vgg_16/conv4/conv4_3'] = (8,8)

In [ ]:
net = decoder.build(tensors_to_connect)

## Train

In [ ]:
trainer = Trainer(nb_classes=2, optimizer=tf.train.AdamOptimizer, learning_rate=1e-4)

In [ ]:
trainer.build(predictions=net, labels=label,decoder_scope='decoder')

In [ ]:
trainer.train(iterator,['data/kitti_segmentation.tfrecord'])